In [1]:
#import packages


import time
import re
import logging
import warnings
import nltk
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
from gensim.models import word2vec
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [2]:
nltk.download('popular')
warnings.filterwarnings('ignore')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

In [7]:
path_train_data = 'labeledTrainData.tsv'

df_test = pd.read_csv(path_train_data, delimiter = '\t')

In [4]:
#Suppression des balises HTML 
'''
remove_balise_html(html_doc, parser = 'html.parser')
remove html balises on an str file
args : 
    - html_doc = html document to clean
    - parser
'''


def remove_balise_html(html_doc,parser='html.parser'):
    cleantext = BeautifulSoup(html_doc, parser).text
    return cleantext

In [5]:
'''
function to clean a list containing the words of the sentence.
- Replace capitals letters
- remove punctuation
- remove digits

arg : 
    t : a list of strings 
'''


def clean_txt(t):
    
    for i in range(len(t)) :
        
        t[i] = t[i].lower()
        t[i] = re.sub(r'[^\w\s]','',t[i])
        t[i] = ''.join([j for j in t[i] if not j.isdigit()])
        
    return t 

In [6]:
lemmatizer = WordNetLemmatizer()

In [8]:
df_test['review'] = df_test['review'].apply(remove_balise_html)
df_test['review'] = df_test['review'].apply(lambda row : remove_stopwords(row))
df_test['review'] = df_test['review'].apply(word_tokenize)
df_test['review'] = df_test['review'].apply(lambda row : clean_txt(row))
df_test['review'] = df_test['review'].apply(lambda row : list(filter(None,row))) #retrait des éléments vides
df_test['review'] = df_test['review'].apply(lambda row : ' '.join([lemmatizer.lemmatize(w) for w in row]))
df_test['review'] = df_test['review'].apply(lambda row : row.split())

In [12]:
model = word2vec.Word2Vec.load("My_Word2Vec")


In [13]:
def get_vect(word, model):
    try:
        return model.wv[word]
    except KeyError:
        return np.zeros((model.vector_size,))

def sum_vectors(phrase, model):
    return sum(get_vect(w, model) for w in phrase)

def word2vec_features(X, model):
    feats = np.vstack([sum_vectors(p, model) for p in X])
    return feats

In [14]:
wv_train_feat = word2vec_features(df_test["review"], model)
wv_train_feat.shape

(25000, 300)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(wv_train_feat, 
                                                    df_test['sentiment'], 
                                                    test_size=0.2, 
                                                    random_state=42)

In [16]:
clfwv = LogisticRegression(solver='lbfgs', max_iter=1000)
clfwv.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [17]:
clfwv.score(X_test, y_test)

0.877